In [25]:
import numpy as np
import random
import cv2

In [61]:
img = cv2.imread('1.jpg')
#img = img.astype(np.int64)
#print(img.dtype)

In [2]:
#饱和函数  
def calc_saturation(diff,slope,limit):  
        ret = diff * slope  
        if ret > limit:  
            ret = limit  
        elif (ret < (-limit)):  
            ret = -limit  
        return ret  

def automatic_color_equalization(nimg, slope=10, limit=1000, samples=500):  
 
        nimg = nimg.transpose(2, 0, 1)  

        #Convert input to an ndarray with column-major memory order(仅仅是地址连续，内容和结构不变)  
        nimg = np.ascontiguousarray(nimg, dtype=np.uint8)  

        width=nimg.shape[2]  
        height=nimg.shape[1]  

        cary=[]  

        #随机产生索引  
        for i in range(0,samples):  
            _x=random.randint(0,width)%width  
            _y=random.randint(0,height)%height  

            dict={"x":_x,"y":_y}  
            cary.append(dict)  


        mat=np.zeros((3,height,width),float)  

        r_max = sys.float_info.min  
        r_min = sys.float_info.max  

        g_max = sys.float_info.min  
        g_min = sys.float_info.max  

        b_max = sys.float_info.min  
        b_min = sys.float_info.max  

        for i in range(height):  
            for j in range(width):  
                r=nimg[0,i,j]  
                g=nimg[1,i,j]  
                b=nimg[2,i,j]  

                r_rscore_sum = 0.0  
                g_rscore_sum = 0.0  
                b_rscore_sum = 0.0  
                denominator = 0.0  
            
            for _dict in cary:  
                _x=_dict["x"] #width  
                _y=_dict["y"] #height  

                #计算欧氏距离  
                dist=np.sqrt(np.square(_x-j)+np.square(_y-i))  
                if (dist < height / 5):  
                    continue;  

                _sr=nimg[0,_y,_x]  
                _sg=nimg[1,_y,_x]  
                _sb=nimg[2,_y,_x]  

                r_rscore_sum +=calc_saturation(int(r) - int(_sr),slope,limit) / dist  
                g_rscore_sum +=calc_saturation(int(g) - int(_sg),slope,limit) / dist  
                b_rscore_sum +=calc_saturation(int(b) - int(_sb),slope,limit) / dist  

                denominator += limit / dist  

                r_rscore_sum = r_rscore_sum / denominator  
                g_rscore_sum = g_rscore_sum / denominator  
                b_rscore_sum = b_rscore_sum / denominator  

                mat[0,i,j]=r_rscore_sum  
                mat[1,i,j]=g_rscore_sum  
                mat[2,i,j]=b_rscore_sum  

                if r_max<r_rscore_sum:  
                    r_max=r_rscore_sum  
                if r_min>r_rscore_sum:  
                    r_min=r_rscore_sum  

                if g_max<g_rscore_sum:  
                    g_max=g_rscore_sum  
                if g_min>g_rscore_sum:  
                    g_min=g_rscore_sum  

                if b_max<b_rscore_sum:  
                    b_max=b_rscore_sum  
                if b_min>b_rscore_sum:  
                    b_min=b_rscore_sum  
  
        for i in range(height):  
            for j in range(width):  
                nimg[0, i, j] = (mat[0, i, j] - r_min) * 255 / (r_max - r_min)  
                nimg[1, i, j] = (mat[1, i, j] - g_min) * 255 / (g_max - g_min)  
                nimg[2, i, j] = (mat[2, i, j] - b_min) * 255 / (b_max - b_min)  
        return nimg.transpose(1, 2, 0).astype(np.uint8)

In [187]:
import cv2
import numpy as np
import math
 
def stretchImage(data, s=0.005, bins = 2000):    #线性拉伸，去掉最大最小0.5%的像素值，然后线性拉伸至[0,1]
    ht = np.histogram(data, bins);
    d = np.cumsum(ht[0])/float(data.size)
    lmin = 0; lmax=bins-1
    while lmin<bins:
        if d[lmin]>=s:
            break
        lmin+=1
    while lmax>=0:
        if d[lmax]<=1-s:
            break
        lmax-=1
    return np.clip((data-ht[1][lmin])/(ht[1][lmax]-ht[1][lmin]), 0,1)
 
g_para = {}
def getPara(radius = 5):                        #根据半径计算权重参数矩阵
    global g_para
    m = g_para.get(radius, None)
    if m is not None:
        return m
    size = radius*2+1
    m = np.zeros((size, size))
    for h in range(-radius, radius+1):
        for w in range(-radius, radius+1):
            if h==0 and w==0:
                continue
            m[radius+h, radius+w] = 1.0/math.sqrt(h**2+w**2)
    m /= m.sum()
    g_para[radius] = m
    return m
 
def zmIce(I, ratio=4, radius=300):                     #常规的ACE实现
    para = getPara(radius)
    height,width = I.shape[0:2]
    zh, zw = [0]*radius + list(range(height)) + [height-1]*radius, [0]*radius + list(range(width))  + [width -1]*radius
    Z = I[np.ix_(zh, zw)]
    res = img+0
    for h in range(radius*2+1):
        for w in range(radius*2+1):
            if para[h][w] == 0:
                continue
            res = res + (para[h][w] * np.clip((I-Z[h:h+height, w:w+width])*ratio, -1, 1))
    return res
 
def zmIceFast(I, ratio, radius):                #单通道ACE快速增强实现
    height, width = I.shape[:2]
    if min(height, width) <=2:
        return np.zeros(I.shape)+0.5
    Rs = cv2.resize(I, (int((width+1)/2), int((height+1)/2)))
    Rf = zmIceFast(Rs, ratio, radius)             #递归调用
    Rf = cv2.resize(Rf, (width, height))
    Rs = cv2.resize(Rs, (width, height))
 
    return Rf+zmIce(I,ratio, radius)-zmIce(Rs,ratio,radius)    
            
def zmIceColor(I, ratio=4, radius=3):               #rgb三通道分别增强，ratio是对比度增强因子，radius是卷积模板半径
    res = np.zeros(I.shape)
    for k in range(3):
        res[:,:,k] = stretchImage(zmIceFast(I[:,:,k], ratio, radius))
    return res
 
if __name__ == '__main__':
    m = zmIceColor(img/255.0)*255
    cv2.imwrite('zmIce.jpg', m)

ValueError: operands could not be broadcast together with shapes (1000,1334,3) (4,6) 

In [49]:
stretchImage(img)

array([[[0.01575718, 0.06696802, 0.03545366],
        [0.01969648, 0.07090732, 0.03939295],
        [0.03151436, 0.06696802, 0.05515014],
        ...,
        [0.01575718, 0.01969648, 0.01181789],
        [0.01969648, 0.02363577, 0.01575718],
        [0.01969648, 0.02363577, 0.01575718]],

       [[0.02757507, 0.07090732, 0.03939295],
        [0.02757507, 0.07090732, 0.03939295],
        [0.02757507, 0.06696802, 0.04333225],
        ...,
        [0.01575718, 0.01969648, 0.01181789],
        [0.01969648, 0.02363577, 0.01575718],
        [0.01969648, 0.02363577, 0.01575718]],

       [[0.04333225, 0.07878591, 0.03939295],
        [0.03545366, 0.06696802, 0.03939295],
        [0.03151436, 0.06302873, 0.03545366],
        ...,
        [0.02757507, 0.01969648, 0.01969648],
        [0.02757507, 0.01969648, 0.01969648],
        [0.02757507, 0.01969648, 0.01969648]],

       ...,

       [[0.01575718, 0.01575718, 0.01575718],
        [0.01575718, 0.01575718, 0.01575718],
        [0.01181789, 0

In [188]:
zmIce(img)

KeyboardInterrupt: 

In [ ]:
height,width = img.shape[0:2]
zh, zw = [0]*300 + list(range(height)) + [height-1]*300, [0]*300 + list(range(width))  + [width -1]*300
Z = img[np.ix_(zh, zw)]
res = img*0
print(res.shape)
para = getPara(300)
for h in range(300*2+1):
    for w in range(300*2+1):
        if para[h][w] == 0:
            continue
        res = res + (para[h][w] * np.clip(((img-Z[h:h+height, w:w+width])*4), -1, 1))

(1000, 1334, 3)


In [ ]:
zmIceFast(img, 4, 300)

In [158]:
height,width = img.shape[0:2]
width

1334

In [176]:
Rs = cv2.resize(img, (int((1334+1)/2), int((1000+1)/2)))

In [171]:
int((1334+1)/2)

667

In [174]:
int((1000+1)/2)

500

In [184]:
Rs = cv2.resize(img, (667, 500))
Rs.shape
Rf = zmIceFast(Rs, 4, 300)

ValueError: operands could not be broadcast together with shapes (4,6,3) (2,) 